In [1]:
import pandas as pd
import json
import requests
import media_mapper

read a json file of tweets for experimentation:

In [2]:
def json_to_df(filename):
    '''Takes in a json file. Returns a pandas dataframe'''
    with open('tweets.json', 'r') as f:
        l = f.readlines()

    data = [json.loads(s) for s in l]
    return pd.DataFrame(data)

get pertinent columns

In [13]:
original_df = json_to_df('tweets.json')


In [4]:
def extract_columns(df):
    '''Takes in a pandas dataframe. Returns a smaller dataframe:
    Text, Coordinates, timestamp'''
    #only include rows with coordinates
    df = df[~df.coordinates.isnull()]
    #make a new dataframe with coordinates, tweets, and timestamps
    df = df[['coordinates', 'text' ,'timestamp_ms']]
    #get a list of coordinates to break it into long and lat data
    coor = df.coordinates.tolist()
    #list of the longitude coordinates
    lons = [c['coordinates'][0] for c in coor]
    #list of the latitude coordinates 
    lats = [c['coordinates'][1] for c in coor]
    
    #turn lats and longs into panda series. Append them to the dataframe.
    df['lons'] = pd.Series(lons)
    df['lats'] = pd.Series(lats)
    df.drop('coordinates', 1)
    
    return df 

In [5]:
df = extract_columns(original_df)

In [6]:
df.head()

,coordinates,text,timestamp_ms,lons,lats
41,"{u'type': u'Point', u'coordinates': [-121.799,...","Ok, it says something else on the glass, but I...",1436937112887,-122.050000,37.016667
45,"{u'type': u'Point', u'coordinates': [-122.4055...",Closed Street or Sidewalk Cleaning request at ...,1436937114489,-123.012313,37.699791
46,"{u'type': u'Point', u'coordinates': [-122.0565...",being with family makes my heart so happy❤,1436937115452,-122.398488,37.789027
51,"{u'type': u'Point', u'coordinates': [-116.9142...",@joannacast_x3 @leilashmeila my dad 😂😂😂😂,1436937117182,-121.887759,37.334962
54,"{u'type': u'Point', u'coordinates': [-121.8991...",Vanilla bean frappe to pair with Papa P! 😍 th...,1436937118044,-122.402552,37.784495


Get Block Labels from the Census:

In [15]:
def get_census_label(df):
    '''Takes in my standard dataframe from the Json object.
    Returns lats and longs to poll sensus for '''
    #select rows that have coordinates
    df = df[~df.coordinates.isnull()]
    #make a new dataframe with coordinates, tweets, and timestamps
    df = df[['coordinates', 'text' ,'timestamp_ms']]
    
    #get a list of coordinates to break it into long and lat data
    coor = df.coordinates.tolist()
    #list of the longitude coordinates
    lons = [c['coordinates'][0] for c in coor]
    #list of the latitude coordinates 
    lats = [c['coordinates'][1] for c in coor]
    return lats, lons

In [16]:
lats, lons = get_census_label(original_df)

In [28]:
def query_census_slowly(lats, lons):
    results = []
    for i in xrange(len(lats)):
        lattitude = str(lats[i])
        longitude = str(lons[i])
        result = requests.get('http://data.fcc.gov/api/block/find?format=json&latitude='+lattitude+'&longitude='+longitude+'&showall=false').json()
        results.append(result['Block']['FIPS'])
    return results

In [ ]:
results = query_census_slowly(lats, lons)

[u'060855033153011', u'060750106002002', u'060014352001012']

In [ ]:
returntype = #locations or geographies? 
searchtype = #onlineaddress OR address OR coordinates
parameters = 

##Easy Way to Turn Coordinates into Blocks / Categories

ask census for information!
[GeocodingCensus](http://www.census.gov/data/developers/data-sets/Geocoding-services.html)

In [ ]:
resp = requests.get(http://geocoding.geo.census.gov/geocoder/returntype/searchtype?parameters&format=json)
        

example with john's request:

In [9]:
result = requests.get('http://data.fcc.gov/api/block/find?format=json&latitude='+'37.016667'+'&longitude='+'-122.050000'+'&showall=false').json()
print result

{u'County': {u'FIPS': u'06087', u'name': u'Santa Cruz'}, u'status': u'OK', u'State': {u'code': u'CA', u'FIPS': u'06', u'name': u'California'}, u'Block': {u'FIPS': u'060871207003033'}, u'executionTime': u'163'}


In [11]:
result = requests.get('http://data.fcc.gov/api/block/find?format=json&latitude='+'37.016667'+'&longitude='+'-122.050000'+'&showall=false').json()
print result['Block']['FIPS']

060871207003033


###registered on Mapbox
drxtwilob, attention shhh

###random notes from john about census

requests direct it at the census api. geocoding services
requests.get(http://data.fcc.gov.... +.json)format=json find lat long
so can use this to get this 
XXstateXXXXXXcountyXXblock
they have examples to do the full street address
geocoding services api
base - http://api.census.gov/data
    
    number of peopel really accurately. could be really interesting
there are two surveys: the acs, or sf
    
    will need an api key: api cencus key
state county tract blockgroup
came out as unicodes.  dtype=object can force it into the correct datatype
he loaded the text with json 